In [75]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv("../../data/processed/data_2.csv",
                   dtype = {
                            "expensa_id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                            "unidad_denominacion":"object",
                            "consorcio_nombre": "object",
                            "version":"object",
                            "concepto_id": "object",
                            "unidad_tipo": "object",
                            "modo_pago": "object",
                            "forma_prorrateo": "object",
                            "tipo_prorrateo": "object",
                            "metodo_pago": "object",    
                            "venc_1_dayofweek": "object",
                            "venc_2_dayofweek": "object",
                            "venc_1_dayofweek_ant": "object",
                            "venc_2_dayofweek_ant": "object",
                            "dia_inicio_mes_vencimiento": "object"
                    },
                    parse_dates = ["fecha_vencimiento_1", "fecha_vencimiento_2"])

data = data[['unidad_id', 'unidad_tipo', 'modo_pago', 'tamanio_consorcio',
             'fecha_expensa_x','mes_vencimiento','dia_inicio_mes_vencimiento',
             'fecha_vencimiento_1', 'venc_1_dayofweek', 'venc_1_day',
             'fecha_vencimiento_2', 'venc_2_dayofweek', 'venc_2_day',             
             'monto', 'monto_total', 'monto_parcial', 
             'int_dia', 'int_mes', 
             'cum_impago_ant', 'cum_pago_tardio_1_ant','cum_pago_tardio_2_ant',
             'mes',  
             'version',
             'metodo_pago',
             'impago', 'pago_tardio_1', 'pago_tardio_2',
             'fecha_expensa_anterior']]

data.columns = ['unidad_id', 'unidad_tipo', 'modo_pago', 'tamanio_consorcio',
                 'fecha_expensa','mes_vencimiento','dia_inicio_mes_vencimiento',
                 'fecha_vencimiento_1', 'fecha_vencimiento_1_dayofweek', 'fecha_vencimiento_1_daynumber',
                 'fecha_vencimiento_2', 'fecha_vencimiento_2_dayofweek', 'fecha_vencimiento_2_daynumber',             
                 'monto', 'monto_total', 'monto_parcial', 
                 'int_dia', 'int_mes', 
                 'cantidad_impagos_a_la_fecha', 'cantidad_pagos_tardios_1_a_la_fecha','cantidad_pagos_tardios_2_a_la_fecha',
                 'mes',  
                 'version',        
                 'metodo_pago',
                 'impago', 'pago_tardio_1', 'pago_tardio_2',
                 'fecha_expensa_anterior']

data["target"] = data["impago"].map(int).map(str) + data["pago_tardio_1"].map(int).map(str) +data["pago_tardio_2"].map(int).map(str)

data.loc[data.target == '000', "target"] = "000"
data.loc[data.target == '100', "target"] = "100"
data.loc[data.target == '010', "target"] = "110"
data.loc[data.target == '001', "target"] = "111"
data = data.set_index(["unidad_id","fecha_expensa"])

In [76]:
data_mes_anterior = data[['target', 'fecha_expensa_anterior']]

In [77]:
data_lstm = data.reset_index()[["unidad_id","fecha_expensa","target", "fecha_expensa_anterior"]]

In [78]:
data_lstm = pd.merge(data_lstm,
         data_mes_anterior,
         left_on = ["unidad_id","fecha_expensa_anterior"],
         right_index = True,
         suffixes = ("", "_lag_1"))

for i in range(1,6):
    data_lstm = pd.merge(data_lstm,
                         data_mes_anterior,
                         left_on = ["unidad_id","fecha_expensa_anterior_lag_{}".format(i)],
                         right_index = True,
                         suffixes = ("", "_lag_{}".format(i+1)))

In [79]:
target_columns = ["fecha_expensa"] + [column for column in data_lstm.columns if column[0:6] == "target"]

In [80]:
data_lstm = data_lstm[target_columns]

In [81]:
data_lstm.shape

(213177, 8)

In [82]:
X = data_lstm.drop(columns="target")
Y = data_lstm[["fecha_expensa", "target"]]

In [83]:
X = X.loc[data_lstm.fecha_expensa < "2019-01-01",:]
Y = Y.loc[data_lstm.fecha_expensa < "2019-01-01",:]

In [84]:
print(X.shape)
print(Y.shape)

(187658, 7)
(187658, 2)


In [85]:
X.groupby([column for column in X.columns if column[0:6] == "target"]).count().reset_index().sort_values("fecha_expensa", ascending=False)

,target_lag_1,target_lag_2,target_lag_3,target_lag_4,target_lag_5,target_lag_6,fecha_expensa
0,000,000,000,000,000,000,47915
2237,111,111,111,111,111,111,12326
826,100,100,100,100,100,100,10357
2,000,000,000,000,000,110,4155
8,000,000,000,000,110,000,3160
1179,110,000,000,000,000,000,3032
28,000,000,000,110,000,000,2664
318,000,110,000,000,000,000,2650
96,000,000,110,000,000,000,2475
1590,110,110,110,110,110,110,2324


In [86]:
target_columns2 = [column for column in X.columns if column[0:6] == "target"]
X_train = X.loc[X.fecha_expensa < '2018-09-01', target_columns2]
X_test = X.loc[X.fecha_expensa >= '2018-09-01', target_columns2]
Y_train = Y.loc[Y.fecha_expensa < '2018-09-01', "target"]
Y_test =Y.loc[Y.fecha_expensa >= '2018-09-01', "target"]
Y_test_2 = Y.loc[Y.fecha_expensa >= '2018-09-01', "target"]
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(139028, 6) (48630, 6)
(139028,) (48630,)


In [87]:
X_train = X_train.iloc[28:,:]
X_test = X_test.iloc[:-30,:]
Y_train = Y_train.iloc[28:]
Y_test = Y_test.iloc[:-30]
Y_test_2 = Y_test_2.iloc[:-30]

In [14]:
# /X_train = pd.get_dummies(X_train).values.reshape(X_train.shape[0],12,4)
# X_test = pd.get_dummies(X_test).values.reshape(X_test.shape[0],12,4)

In [88]:
def convert_to_ordinal_classes(df):
    for column in df.columns:
        df["N_" + column + "_PT1"] = df[column].str[0].map(int)
        df["N_" + column + "_PT2"] = df[column].str[1].map(int)
        df["N_" + column + "_NP"] = df[column].str[2].map(int)

    return df.iloc[:,6:].values.reshape(df.shape[0],6,3)
    

In [89]:
X_train = convert_to_ordinal_classes(X_train)
X_test = convert_to_ordinal_classes(X_test)

In [91]:
def convert_to_ordinal_classes_target(serie):
    df = pd.DataFrame({"target": serie})
    for column in df.columns:
        df["N_" + column + "_PT1"] = df[column].str[0].map(int)
        df["N_" + column + "_PT2"] = df[column].str[1].map(int)
        df["N_" + column + "_NP"] = df[column].str[2].map(int)

    return df.iloc[:,1:].values.reshape(df.shape[0],3)
Y_train = convert_to_ordinal_classes_target(Y_train)
Y_test = convert_to_ordinal_classes_target(Y_test)

In [ ]:
# X_train = np.array(X_train.values).reshape(*X_train.shape)
# X_test = np.array(X_test.values).reshape(*X_test.shape)

# Y_train = np.array(pd.get_dummies(Y_train)[[0,1,2,3]].values).reshape(Y_train.shape[0],4)
# Y_test = np.array(pd.get_dummies(Y_test)[[0,1,2,3]].values).reshape(Y_test.shape[0],4)

In [92]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(139000, 6, 3) (48600, 6, 3)
(139000, 3) (48600, 3)


In [93]:
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.layers import LSTM, Embedding

lstm_model = Sequential()
lstm_model.add(LSTM(128, batch_input_shape=(100,6,3), dropout=0.0, recurrent_dropout=0.0, stateful=True, kernel_initializer='random_uniform'))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(64,activation='relu'))
lstm_model.add(Dense(3,activation='sigmoid'))
lstm_model.compile(loss='categorical_crossentropy', optimizer="adam")

In [94]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (100, 128)                67584     
_________________________________________________________________
dropout_5 (Dropout)          (100, 128)                0         
_________________________________________________________________
dense_9 (Dense)              (100, 64)                 8256      
_________________________________________________________________
dense_10 (Dense)             (100, 3)                  195       
Total params: 76,035
Trainable params: 76,035
Non-trainable params: 0
_________________________________________________________________


In [95]:
lstm_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=40, verbose=2, batch_size=100,
                    shuffle=False)

Train on 139000 samples, validate on 48600 samples
Epoch 1/40
 - 19s - loss: 1.0416 - val_loss: 1.0073
Epoch 2/40
 - 19s - loss: 1.0007 - val_loss: 0.9312
Epoch 3/40
 - 24s - loss: 0.9991 - val_loss: 0.9239
Epoch 4/40
 - 24s - loss: 1.0020 - val_loss: 0.9335
Epoch 5/40
 - 23s - loss: 0.9960 - val_loss: 0.9125
Epoch 6/40
 - 26s - loss: 0.9946 - val_loss: 0.9064
Epoch 7/40
 - 25s - loss: 0.9941 - val_loss: 0.9053
Epoch 8/40
 - 23s - loss: 0.9927 - val_loss: 0.9039
Epoch 9/40
 - 25s - loss: 0.9912 - val_loss: 0.9032
Epoch 10/40
 - 24s - loss: 0.9903 - val_loss: 0.9037
Epoch 11/40
 - 27s - loss: 0.9898 - val_loss: 0.9025
Epoch 12/40
 - 24s - loss: 0.9895 - val_loss: 0.9046
Epoch 13/40
 - 24s - loss: 0.9885 - val_loss: 0.9032
Epoch 14/40
 - 25s - loss: 0.9888 - val_loss: 0.9017
Epoch 15/40
 - 25s - loss: 0.9872 - val_loss: 0.9035
Epoch 16/40
 - 26s - loss: 0.9870 - val_loss: 0.9008
Epoch 17/40
 - 30s - loss: 0.9866 - val_loss: 0.9016
Epoch 18/40
 - 24s - loss: 0.9874 - val_loss: 0.9025
Epoc

In [240]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
confusion_matrix(Y_test_2, lstm_model.predict_classes(X_test, batch_size=100))

array([[25907,   119,    37,  2053],
       [  461,  3821,     8,   410],
       [ 5528,    65,    48,  1430],
       [ 3601,    55,    25,  5032]], dtype=int64)

In [203]:
pred = lstm_model.predict_classes(X_test, batch_size=100)

In [204]:
confusion_matrix(Y_test_2, pred)

array([[18672,    87,     3,  1373],
       [  414,  2177,     1,   374],
       [ 4030,    61,     2,  1224],
       [ 2200,    35,     0,  3247]], dtype=int64)

In [105]:
def get_array_of_clases(array, threshold = 0.5):
    classes_array = np.ndarray((array.shape[0], 1))
    for i in range(0,array.shape[0]):
        for j in range(0,array.shape[1]):
            if array[i,j] < threshold:
                classes_array[i] = j
                break
            classes_array[i] = j + 1
    return classes_array

In [106]:
list(reversed(range(0, Y_test.shape[1])))

[2, 1, 0]

In [107]:
clases = get_array_of_clases(Y_test)

In [108]:
Y_test[678]

array([1, 1, 0], dtype=int64)

In [109]:
clases[678]

array([2.])

In [110]:
predictions = lstm_model.predict(X_test, batch_size=100)

In [112]:
get_array_of_clases(predictions)

array([[2.],
       [2.],
       [2.],
       ...,
       [2.],
       [2.],
       [2.]])

In [74]:
Y_test[1:10]

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 1, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=int64)

In [117]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
real = get_array_of_clases(Y_test)
predicted = get_array_of_clases(lstm_model.predict(X_test, batch_size=100), 0.90)
confusion_matrix(real, predicted)

array([[ 2031,  1473, 24358,   254],
       [   50,  3770,   725,   155],
       [  197,   281,  6375,   218],
       [   98,   115,  7042,  1458]], dtype=int64)